In [89]:
#Import Dependencies

%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import os
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import session
from sqlalchemy import create_engine, func
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
import tensorflow as tf

In [90]:
#SOURCE: https://www.tutorialspoint.com/python_data_access/python_postgresql_database_connection.htm
import psycopg2

#establishing the connection
conn = psycopg2.connect(
   database="Body_Performance", user='postgres', password='Cheese6132', host='localhost', port= '5432'
)
#Creating a cursor object using the cursor() method
cursor = conn.cursor()

#Executing an MYSQL function using the execute() method
cursor.execute("select version()")

# Fetch a single row using fetchone() method.
data = cursor.fetchone()
print("Connection established to: ",data)



Connection established to:  ('PostgreSQL 12.13, compiled by Visual C++ build 1914, 64-bit',)


In [91]:
body_df = pd.read_sql('select * from body_performance', con=conn)
body_df.head()

,age,gender,height_cm,weight_kg,body_fat_pct,diastolic,systolic,gripforce,sit_and_bend_forward_cm,sit_ups_counts,broad_jump_cm,ranking
0,27,M,172.3,75.24,21.3,80.0,130.0,54.9,18.4,60.0,217.0,C
1,25,M,165.0,55.80,15.7,77.0,126.0,36.4,16.3,53.0,229.0,A
2,31,M,179.6,78.00,20.1,92.0,152.0,44.8,12.0,49.0,181.0,C
3,32,M,174.5,71.10,18.4,76.0,147.0,41.4,15.2,53.0,219.0,B
4,28,M,173.8,67.70,17.1,70.0,127.0,43.5,27.1,45.0,217.0,B


In [92]:
#Closing the connection
conn.close()
print("Closed connection established to: ",data)


Closed connection established to:  ('PostgreSQL 12.13, compiled by Visual C++ build 1914, 64-bit',)


In [93]:
# OneHotEncoder or Get_dummies on Gender
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(sparse=False)

encode1 = pd.DataFrame(enc.fit_transform(body_df.gender.values.reshape(-1,1)))

encode1.columns = enc.get_feature_names_out(['gender'])
encode1.head()


,gender_F,gender_M
0,0.0,1.0
1,0.0,1.0
2,0.0,1.0
3,0.0,1.0
4,0.0,1.0


In [ ]:
#If keeping class/level in one column as Y, Encode with dictionary values
#ranking_num = {"A":0, "B":1, "C":2, "D":3}
#body_df["ranking_num"] = body_df["ranking"].apply(lambda x: ranking_num[x])
#body_df.head()


In [94]:
#Using oneHotEncoder for class into 4 separate

encode2 = pd.DataFrame(enc.fit_transform(body_df.ranking.values.reshape(-1,1)))

encode2.columns = enc.get_feature_names_out(["ranking"])
encode2.head()

,ranking_A,ranking_B,ranking_C,ranking_D
0,0.0,0.0,1.0,0.0
1,1.0,0.0,0.0,0.0
2,0.0,0.0,1.0,0.0
3,0.0,1.0,0.0,0.0
4,0.0,1.0,0.0,0.0


In [95]:
#Drop columns and merge encoded df
#Ignoring encode2 for now
body_df = body_df.drop(["gender", "ranking"], axis=1)

body_df2 = body_df.merge(encode1, left_index=True, right_index=True)
body_df2.head()


,age,height_cm,weight_kg,body_fat_pct,diastolic,systolic,gripforce,sit_and_bend_forward_cm,sit_ups_counts,broad_jump_cm,gender_F,gender_M
0,27,172.3,75.24,21.3,80.0,130.0,54.9,18.4,60.0,217.0,0.0,1.0
1,25,165.0,55.80,15.7,77.0,126.0,36.4,16.3,53.0,229.0,0.0,1.0
2,31,179.6,78.00,20.1,92.0,152.0,44.8,12.0,49.0,181.0,0.0,1.0
3,32,174.5,71.10,18.4,76.0,147.0,41.4,15.2,53.0,219.0,0.0,1.0
4,28,173.8,67.70,17.1,70.0,127.0,43.5,27.1,45.0,217.0,0.0,1.0


In [97]:
#Set X, y 
X = body_df2.values
    #X = X.drop("level_num", axis=1)
    #y = body_df2["level_num"].ravel()
y = encode2.values
print(X.shape, y.shape)


(13393, 12) (13393, 4)


In [98]:
y[:10]

array([[0., 0., 1., 0.],
       [1., 0., 0., 0.],
       [0., 0., 1., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 0., 1.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 1., 0., 0.]])

In [99]:
#Split train/test data and scale using Standard Scaler at this time (mean of 0).  #Should we use MinMax(0-1)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

scaler = StandardScaler().fit(X_train)

X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)
print(X_train_scaled.shape, X_test_scaled.shape)

(10044, 12) (3349, 12)


In [100]:
# Define the model - deep neural net...length of outputs/inputs
num_input_features = len(X_train_scaled[0])
num_output_features = len(y_train[0])
print(num_input_features)
print(num_output_features)

12
4


In [166]:
# Define the model - deep neural net
# Try with 2 hidden layers first but will try reduced. Look at different activation functions?
hidden_nodes_layer1 =  24
hidden_nodes_layer2 = 18
hidden_nodes_layer3 = 12

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(
    units=hidden_nodes_layer1, input_dim=num_input_features,
    activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="sigmoid"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="sigmoid"))

# Output layer  #Will need to round or should this be sigmoid?
nn.add(tf.keras.layers.Dense(units=num_output_features, activation="softmax"))

# Check the structure of the model
nn.summary()

Model: "sequential_28"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_83 (Dense)            (None, 24)                312       
                                                                 
 dense_84 (Dense)            (None, 18)                450       
                                                                 
 dense_85 (Dense)            (None, 12)                228       
                                                                 
 dense_86 (Dense)            (None, 4)                 52        
                                                                 
Total params: 1,042
Trainable params: 1,042
Non-trainable params: 0
_________________________________________________________________


In [167]:
# Compile the model binary vs categorical. Categorical for multi-output  --https://machinelearningmastery.com/deep-learning-models-for-multi-output-regression/
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
#nn.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

In [170]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=200)

Epoch 1/200
314/314 [==============================] - 0s 786us/step - loss: 0.2594 - accuracy: 0.7667
Epoch 2/200
314/314 [==============================] - 0s 819us/step - loss: 0.2598 - accuracy: 0.7694
Epoch 3/200
314/314 [==============================] - 0s 801us/step - loss: 0.2598 - accuracy: 0.7659
Epoch 4/200
314/314 [==============================] - 0s 782us/step - loss: 0.2598 - accuracy: 0.7688
Epoch 5/200
314/314 [==============================] - 0s 781us/step - loss: 0.2592 - accuracy: 0.7668
Epoch 6/200
314/314 [==============================] - 0s 778us/step - loss: 0.2591 - accuracy: 0.7689
Epoch 7/200
314/314 [==============================] - 0s 781us/step - loss: 0.2595 - accuracy: 0.7654
Epoch 8/200
314/314 [==============================] - 0s 773us/step - loss: 0.2588 - accuracy: 0.7678
Epoch 9/200
314/314 [==============================] - 0s 775us/step - loss: 0.2590 - accuracy: 0.7672
Epoch 10/200
314/314 [==============================] - 0s 784us/step - l

314/314 [==============================] - 0s 775us/step - loss: 0.2522 - accuracy: 0.7712
Epoch 80/200
314/314 [==============================] - 0s 773us/step - loss: 0.2523 - accuracy: 0.7712
Epoch 81/200
314/314 [==============================] - 0s 771us/step - loss: 0.2519 - accuracy: 0.7747
Epoch 82/200
314/314 [==============================] - 0s 771us/step - loss: 0.2526 - accuracy: 0.7712
Epoch 83/200
314/314 [==============================] - 0s 772us/step - loss: 0.2519 - accuracy: 0.7724
Epoch 84/200
314/314 [==============================] - 0s 778us/step - loss: 0.2521 - accuracy: 0.7686
Epoch 85/200
314/314 [==============================] - 0s 775us/step - loss: 0.2516 - accuracy: 0.7744
Epoch 86/200
314/314 [==============================] - 0s 766us/step - loss: 0.2520 - accuracy: 0.7724
Epoch 87/200
314/314 [==============================] - 0s 822us/step - loss: 0.2514 - accuracy: 0.7722
Epoch 88/200
314/314 [==============================] - 0s 774us/step - loss:

314/314 [==============================] - 0s 776us/step - loss: 0.2473 - accuracy: 0.7768
Epoch 158/200
314/314 [==============================] - 0s 876us/step - loss: 0.2471 - accuracy: 0.7770
Epoch 159/200
314/314 [==============================] - 0s 778us/step - loss: 0.2472 - accuracy: 0.7766
Epoch 160/200
314/314 [==============================] - 0s 851us/step - loss: 0.2470 - accuracy: 0.7790
Epoch 161/200
314/314 [==============================] - 0s 789us/step - loss: 0.2469 - accuracy: 0.7774
Epoch 162/200
314/314 [==============================] - 0s 794us/step - loss: 0.2463 - accuracy: 0.7783
Epoch 163/200
314/314 [==============================] - 0s 898us/step - loss: 0.2467 - accuracy: 0.7769
Epoch 164/200
314/314 [==============================] - 0s 879us/step - loss: 0.2469 - accuracy: 0.7777
Epoch 165/200
314/314 [==============================] - 0s 889us/step - loss: 0.2469 - accuracy: 0.7783
Epoch 166/200
314/314 [==============================] - 0s 809us/ste

In [172]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

105/105 - 0s - loss: 0.2910 - accuracy: 0.7372 - 70ms/epoch - 671us/step
Loss: 0.2909781336784363, Accuracy: 0.7372350096702576


In [ ]:
# Define the model - deep neural net
# Try with 1 hidden layer keeping activation functions and node#

nn2 = tf.keras.models.Sequential()

# First hidden layer
nn2.add(tf.keras.layers.Dense(
    units=hidden_nodes_layer1, input_dim=num_input_features,
    activation="relu"))

# Second hidden layer
#nn2.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer  #Will need to round or should this be sigmoid?
nn2.add(tf.keras.layers.Dense(units=4, activation="sigmoid"))

# Check the structure of the model
nn2.summary()

In [ ]:
# Compile the model
nn2.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [ ]:
# Train the model
fit_model = nn2.fit(X_train_scaled,y_train,epochs=100)

In [ ]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn2.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

#One Hidden Layer lowered accuracy and inreased loss. 

In [ ]:
feature_importances = rf_model.feature_importances_
sorted(zip(rf_model.feature_importances_, X.columns), reverse=True)

In [ ]:
#Accuracy score and confusion Matrix
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

predictions = rf_model.predict(X_test_scaled)
acc_score = accuracy_score(y_test, predictions)
print(f"Accuracy Score : {acc_score}")

In [ ]:
# printing confusion matrix but error:      Shape of passed values is (4, 4), indices imply (2, 2) Needs to be resolved
cm = confusion_matrix(y_test, predictions)
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"],
    columns=["Predicted 0", "Predicted 1"])
display(cm_df)

In [ ]:
#Classification report for random forest

print("Classification Report")
print(classification_report(y_test, predictions))

In [ ]:
# Need to test feature importances. THis is one way but plot is not showing up how I'd like. Need to adjust.

features = sorted(zip(X.columns, rf_model.feature_importances_), key = lambda x: x[1])
cols = [f[0] for f in features]
width = [f[1] for f in features]

fig, ax = plt.subplots()

fig.set_size_inches(10,200)
plt.margins(y=0.001)

ax.barh(y=cols, width=width)

plt.show()

In [ ]:
# Binning of Age? if so will need to also be encoded/scaled.

# Open in Google Collab or here and import tensor flow to run as Neural Network

# Try MinMax Scaler  #Should I scale entire 

# Try with multiply Y columns as output. Can I do this with trees or only neural network?